# StyleGAN3

By [Derrick Schultz](https://twitter.com/dvsch), with contributions from [crimeacs](https://twitter.com/EarthML1)

If you find this helpful, please consider backing me on [Patreon](https://www.patreon.com/bustbright) or becoming a [YouTube channel member](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join).

Open the original notebook in colab:

<a href="https://colab.research.google.com/github/dvschultz/stylegan3/blob/main/SG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----

**I only changed some of the settings & data paths for my own use case**

## Setup

In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Uninstall new JAX
!pip uninstall jax jaxlib -y
#GPU frontend
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
#CPU frontend
#!pip install jax[cpu]==0.3.10
#Downgrade Pytorch
!pip uninstall torch torchvision -y
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 imageio-ffmpeg opensimplex

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg3"):
    %cd "/content/drive/MyDrive/colab-sg3/stylegan3/"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg3
    %cd colab-sg3
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install opensimplex

This cell will update to the latest repo. Git and Drive/Colab don’t play as nicely as I’d like so 🤞. The other option is to delete your folder in Drive (after saving out `/results` and `/datasets`!) and running the script above to replace the entire folder.

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py gen_images.py gen_video.py README.md training/training_loop.py

## Convert/Create Dataset
Pass a folder of images (just .pngs? TK) to create a zip file.

In [ ]:
!python dataset_tool.py --source=/content/drive/MyDrive/stylegan/data/sliding_512 --dest=/content/drive/MyDrive/stylegan/data/sliding-512.zip

## Training

Before you start training, read [this](https://github.com/dvschultz/stylegan3/blob/main/docs/configs.md).

Working Notes:
- It appears that you must use an SG3 pre-trained model for transfer learning. I _think_ you also want to match config to the pretrained model (`t` with `t`, `r` with `r`).
- For an `A100` I’ve found you can use a `--batch-gpu=8`. For other GPUs I recommend `--batch-gpu=4`.
- I see `~205 sec/kimg` on A100s, and `~325 sec/kimg` on V100s (1024, `r` config). This seems slightly slower than what [NVIDIA reports.](https://github.com/dvschultz/stylegan3/blob/main/docs/configs.md)

In [ ]:
!python train.py --help

In [ ]:
## Settings for training 512x512
## I changed the configuration, batch-gpu, gamma, data path & resume. I also added metrics & mirrory.
!python train.py --outdir=./results --cfg=stylegan3-r --gpus=1 --batch=32 --batch-gpu=8 --gamma=8 --snap=4 \
--metrics=none --mirror=True --mirrory=True --kimg=1002 --data=/content/drive/MyDrive/stylegan/data/multiscale-512.zip \
--resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl

In [ ]:
## Settings for training 1024x1024
!python train.py --outdir=./results --cfg=stylegan3-t --gpus=1 --batch=32 --batch-gpu=8 --gamma=8 --snap=4 \
--metrics=none --mirror=True --mirrory=True --kimg=1002 --data=/content/drive/MyDrive/stylegan/data/sliding-1024.zip \
--resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl

## Image Generation

In [ ]:
!python gen_images.py --help

In [ ]:
!python gen_images.py --outdir=./out/01_images --trunc=1 --seeds=2-10 \
    --network=/content/drive/MyDrive/colab-sg3/stylegan3/results/00008-stylegan3-r-multiscale-512-gpus1-batch32-gamma8/network-snapshot-000240.pkl

## Video Generation


In [ ]:
!python gen_video.py --help

In [ ]:
## I experimented with different truncations and seeds and added the interpolation and stabilize-video variable
!python gen_video.py --output=/content/lerp.mp4 --trunc=1 --seeds=280,290,300 --grid=1x1 --interpolation=lerp --stabilize-video \
--network=/content/drive/MyDrive/colab-sg3/stylegan3/results/00008-stylegan3-r-multiscale-512-gpus1-batch32-gamma8/network-snapshot-000240.pkl